In [1]:
# Import packages
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np


### Download the data
The original dataset can be found and downloaded from data.gov, or the BRLA landing page provided in the same link.

https://catalog.data.gov/dataset/purchase-orders-and-contracts

Replace the `path` variable with the path to the csv data file.

In [2]:
# Replace with path to csv file
path = ""

df = pd.read_csv(path)

# Filter down to certain expense types
types = ['INVENTORY']
df = df[df['EXPENSE TYPE DESCRIPTION'].isin(types)]

# Drop data with no item descriptions
df = df.dropna(subset=['ITEM DESCRIPTION'])
df = df.reset_index()
df = df.head(500)

# Split into semantic and numerical data
df_desc = df[['DOCUMENT NUMBER', 'INPUT DATE',
       'GEOGRAPHIC AREA', 'DOCUMENT DESCRIPTION', 
       'ITEM DESCRIPTION', 'COMMODITY DESCRIPTION']]
df_num = df[['DOCUMENT NUMBER', 'REQUISITION NUMBER', 'ITEM DESCRIPTION', 'INPUT DATE', 'DEPARTMENT NAME', 'COST CENTER NAME', 'TOTAL ITEMS', 
             'ITEM NUMBER', 'ITEM QUANTITY ORDERED', 'ITEM UNIT OF MEASURE', 'ITEM UNIT COST', 'ITEM TOTAL COST', ]]

# Create a combined string and embeddings for the semantic dataset
embbedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
combined_desc = [str(row["DOCUMENT DESCRIPTION"]) + str(row['ITEM DESCRIPTION']) + str(row['COMMODITY DESCRIPTION']) for index, row in df_desc.iterrows()]
df_desc['combined_desc'] = combined_desc
embeds = [embbedding_model.encode(row['combined_desc']) for index, row in df_desc.iterrows()]
df_desc['embeds'] = [list(x) for x in embeds]

/var/folders/yj/lg41xjmd4gd07_nnl42d2lpm0000gn/T/ipykernel_38988/2539635568.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_desc['combined_desc'] = combined_desc
/var/folders/yj/lg41xjmd4gd07_nnl42d2lpm0000gn/T/ipykernel_38988/2539635568.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_desc['embeds'] = [list(x) for x in embeds]


### Export the data

In [3]:
df_desc.to_csv('./datasets/purchase_order/br_purchase_orders_desc.csv')
df_num.to_csv('./datasets/purchase_order/br_purchase_orders_num.csv')